In [ ]:
# 필요한 라이브러리 로드
import os
import pandas as pd
import xlwings as xw

In [ ]:
# 'adobe_raw' 디렉토리에서 처리할 원본 파일 탐색
# 'New_Weekly_트래픽_Ken'으로 시작하는 파일 사용
for filename in os.listdir('./adobe_raw/'):
    if filename.startswith('New_Weekly_트래픽_Ken'):
        traffic_file = './adobe_raw/' + filename
        break

In [ ]:
# xlwings로 Excel 파일을 열고 데이터 읽기
# DRM 보호 CSV 파일을 읽기 위해 Excel 앱을 직접 제어하는 xlwings 사용
# try...finally 구문으로 Excel 앱의 확실한 종료 보장
workbook = None
try:
    workbook = xw.Book(traffic_file)
    sheet1 = workbook.sheets[0].used_range.value
    df_traffic = pd.DataFrame(sheet1)
finally:
    if workbook:
        workbook.app.quit()

In [ ]:
# 원본 데이터 정제
# 보고서의 헤더, 구분선, 불필요한 행들을 순차적으로 제거
df_cleaned = df_traffic.iloc[9:]
df_cleaned = df_cleaned[df_cleaned[0] != '##############################################']
df_cleaned = df_cleaned.dropna(subset=[0])
df_cleaned = df_cleaned[~df_cleaned[0].isin(['Week', 'Day'])]
df_cleaned = df_cleaned.reset_index(drop=True).copy()

In [ ]:
# 데이터 테이블별 인덱스 식별
title_index = df_cleaned[df_cleaned[0].str.startswith('#', na=False)].index
begin_index = title_index + 1
end_index = title_index[1:] - 1
end_index = end_index.append(pd.Index([len(df_cleaned)]))

In [ ]:
# 딕셔너리를 사용하여 데이터프레임 관리
dataframes = {}
title_list = []

for i_title, i_begin, i_end in zip(title_index, begin_index, end_index):
    df_title_raw = df_cleaned[0][i_title]
    df_title = "df_" + df_title_raw.replace('# ', '').replace(' ','_').replace('전체', 'whole').replace('캠페인', 'campaign').lower()
    title_list.append(df_title)
    dataframes[df_title] = df_cleaned.iloc[i_begin:i_end + 1].copy()

print("생성된 데이터프레임 목록:", title_list)

In [ ]:
# Weekly/Daily 데이터프레임 컬럼 설정
weekly_col_list = [('WeekNumber', ''), ('Visits', 'All'), ('Visits', 'Mobile'), ('Visits', 'PC'), ('UniqueVisitors', 'All'), ('UniqueVisitors', 'Mobile'), ('UniqueVisitors', 'PC'), ('PageViews', 'All'), ('PageViews', 'Mobile'), ('PageViews', 'PC'), ('TimeSpentperVisit', 'All'), ('TimeSpentperVisit', 'Mobile'), ('TimeSpentperVisit', 'PC'), ('Order', 'All'), ('Order', 'Mobile'), ('Order', 'PC'), ('Revenue', 'All'), ('Revenue', 'Mobile'), ('Revenue', 'PC')]
weekly_col = pd.MultiIndex.from_tuples(weekly_col_list)

daily_col_list = [('Date', ''), ('Visits', 'All'), ('Visits', 'Mobile'), ('Visits', 'PC'), ('UniqueVisitors', 'All'), ('UniqueVisitors', 'Mobile'), ('UniqueVisitors', 'PC'), ('PageViews', 'All'), ('PageViews', 'Mobile'), ('PageViews', 'PC'), ('TimeSpentperVisit', 'All'), ('TimeSpentperVisit', 'Mobile'), ('TimeSpentperVisit', 'PC'), ('Order', 'All'), ('Order', 'Mobile'), ('Order', 'PC'), ('Revenue', 'All'), ('Revenue', 'Mobile'), ('Revenue', 'PC')]
daily_col = pd.MultiIndex.from_tuples(daily_col_list)

dataframes['df_whole_weekly'].columns = weekly_col
dataframes['df_campaign_weekly'].columns = weekly_col
dataframes['df_whole_daily'].columns = daily_col
dataframes['df_campaign_daily'].columns = daily_col

In [ ]:
# 'PageType' 컬럼 추가
dataframes['df_whole_weekly']['PageType'] = 'All'
dataframes['df_campaign_weekly']['PageType'] = 'Campaign'
dataframes['df_whole_daily']['PageType'] = 'All'
dataframes['df_campaign_daily']['PageType'] = 'Campaign'

In [ ]:
# Weekly, Daily 데이터 병합
df_campaign_weekly = pd.concat([dataframes['df_whole_weekly'], dataframes['df_campaign_weekly']])
df_campaign_daily = pd.concat([dataframes['df_whole_daily'], dataframes['df_campaign_daily']])

In [ ]:
# Weekly 데이터 Melt (Wide -> Long format)
df_campaign_weekly_melt = pd.melt(
    df_campaign_weekly, 
    id_vars=[('WeekNumber', ''), ('PageType', '')],
    value_vars=weekly_col_list[1:], # WeekNumber 제외
    var_name=['Dimensions', 'Device'],
    value_name='Metrics'
)
df_campaign_weekly_melt.rename(columns={'WeekNumber': 'Date'}, inplace=True)

In [ ]:
# Weekly 데이터 날짜 처리 (ISO Week 기준)
df_campaign_weekly_melt['Date'] = pd.to_datetime(df_campaign_weekly_melt['Date'])
df_campaign_weekly_melt['ISOYear'] = df_campaign_weekly_melt['Date'].dt.isocalendar().year.astype('str')
df_campaign_weekly_melt['ISOWeek'] = df_campaign_weekly_melt['Date'].dt.isocalendar().week.astype('str')
df_campaign_weekly_melt['WeekNumber'] = df_campaign_weekly_melt['ISOYear'] + "-" + df_campaign_weekly_melt['ISOWeek']

# 불필요한 컬럼 제거 및 순서 정리
df_campaign_weekly_melt = df_campaign_weekly_melt.drop(columns=['Date', 'ISOYear', 'ISOWeek'])
df_campaign_weekly_melt = df_campaign_weekly_melt[['WeekNumber', 'PageType', 'Dimensions', 'Device', 'Metrics']]

In [ ]:
# Daily 데이터 Melt (Wide -> Long format)
df_campaign_daily_melt = pd.melt(
    df_campaign_daily, 
    id_vars=[('Date', ''), ('PageType', '')],
    value_vars=daily_col_list[1:], # Date 제외
    var_name=['Dimensions', 'Device'],
    value_name='Metrics'
)
df_campaign_daily_melt.columns = ['Date', 'PageType', 'Dimensions', 'Device', 'Metrics']

In [ ]:
# 데이터 정렬
df_campaign_weekly_melt = df_campaign_weekly_melt.sort_values(by=['WeekNumber', 'PageType', 'Dimensions', 'Device'])
df_campaign_daily_melt = df_campaign_daily_melt.sort_values(by=['Date', 'PageType', 'Dimensions', 'Device'])

In [ ]:
# CSV 파일로 저장
df_campaign_weekly_melt.to_csv('./csv_raw/campaign_weekly.csv', index=False, encoding='utf-8-sig')
df_campaign_daily_melt.to_csv('./csv_raw/campaign_daily.csv', index=False, encoding='utf-8-sig')